In [1]:
from pynq import Overlay
import numpy as np

stream_overlay=Overlay('/home/xilinx/pynq/overlays/project_aes/aes_2.bit')


In [2]:
stream_overlay?

In [3]:
dma = stream_overlay.aes_comb.axi_dma_0

In [4]:
dma?

In [5]:
from pynq import allocate
import numpy as np

puint128 = np.dtype([('upper', np.uint64),('lower', np.uint64)])

d_in = allocate(shape=(1,), dtype = puint128)
d_out = allocate(shape=(1,), dtype = puint128)
d_in[0][0] = 0x0000000000000000
d_in[0][1] = 0x0000000000000000
d_out[0][0] = 0x0000000000000000
d_out[0][1] = 0x0000000000000000
#key[0][0] = 0x54776F204F6E6520
#key[0][1] = 0x4E696E652054776F
#key2 = 0x54776F204F6E65204E696E652054776F

#k0 = 0x54776F20
#k1 = 0x4F6E6520
#k2 = 0x4E696E65
#k3 = 0x2054776F

#k00 = 0x10a58869
#k11 = 0xd74be5a3
#k22 = 0x74cf867c
#k33 = 0xfb473859

#EXAMPLE INPUT KEY:0x5468617473206D79204B756E67204675

tk0 = 0x54686174
tk1 = 0x73206D79
tk2 = 0x204B756E
tk3 = 0x67204675
td_in2 = allocate(shape=(1,), dtype = puint128)
td_out = allocate(shape=(1,), dtype = puint128)

#EXAMPLE INPUT 128-bit TEXT: 0x54776F204F6E65204E696E652054776F
#input with higher bits in second number
td_in2[0][1] = 0x54776F204F6E6520
td_in2[0][0] = 0x4E696E652054776F
td_out[0][0] = 0x0000000000000000
td_out[0][1] = 0x0000000000000000

#EXAMPLE EXPECTED OUTPUT 128-bit TEXT:0x29C3505F571420F6402299B31A02D73A
#print(hex(inarr[0][0]))

#print(puint128.itemsize)

In [7]:
import time
start = time.time()
stream_overlay.aes_comb.aes_encrypt_impl_0.write(0x1c, tk0)
stream_overlay.aes_comb.aes_encrypt_impl_0.write(0x18, tk1)
stream_overlay.aes_comb.aes_encrypt_impl_0.write(0x14, tk2)
stream_overlay.aes_comb.aes_encrypt_impl_0.write(0x10, tk3)
start = time.time()
dma.sendchannel.transfer(td_in2)
dma.recvchannel.transfer(td_out)
dma.sendchannel.wait()
dma.recvchannel.wait()
end = time.time() - start
print(end*1000)


2.5267601013183594


In [8]:
print(hex(td_out[0][0]))
print(hex(td_out[0][1]))

0x402299b31a02d73a
0x29c3505f571420f6
